In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
cwd='/content/drive/MyDrive/tr_ar' #directory where we keep the data
if not os.path.exists(cwd): os.makedirs(cwd)  #if the directory doesn't exist, create it
os.chdir(cwd) #change current work directory to the directory where we keep the data
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/tr_ar


In [2]:
#Clone the github codebase
!rm -r code_utils #first delete the code_utils folder, then clone from the codebase (very crude till we figure out using github from colab)
!git clone https://github.com/hmghaly/word_align.git code_utils

Cloning into 'code_utils'...
remote: Enumerating objects: 875, done.
remote: Counting objects: 100% (689/689), done.
remote: Compressing objects: 100% (656/656), done.
remote: Total 875 (delta 117), reused 534 (delta 24), pack-reused 186
Receiving objects: 100% (875/875), 26.35 MiB | 12.52 MiB/s, done.
Resolving deltas: 100% (218/218), done.


#Word aligment- Phrase extraction pipeline

In [3]:
from code_utils.general import *
from code_utils.align_utils import *
from code_utils.arabic_lib import *

from itertools import groupby
from collections import Counter
ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})

def cur_ar_tok_fn(text):
  cur_toks=tok(text)
  return tok_ar(cur_toks,ar_counter_dict)
print("loaded necessary functions")
#TESTING
# sents=["here is it, I'll tell you","what's this?","When I go, I'll tell you!"]
# sents=["معظم المنظمات والأشخاص العاملين فيها","ونحن في المكتب","وهذه هي المشكلة الأكبر فيها وفي معظم الأمور الأخرى"]
# #test_indexing_obj=indexing(sents)
# test_indexing_obj=indexing(sents,cur_ar_tok_fn,ignore_ar_pre_suf=True)
# print(test_indexing_obj.all_tok_sentences)
# print(test_indexing_obj.all_tok_original_sentences)
# print(test_indexing_obj.all_mappings)

loaded necessary functions


#Loading sentences

In [10]:
ar_corpus_fname="UNv1.0.ar-en-clean2.ar"
en_corpus_fname="UNv1.0.ar-en.en"

ar_fopen=open(ar_corpus_fname)
en_fopen=open(en_corpus_fname)

src_sents,trg_sents=[],[]
for i,pair in enumerate(zip(en_fopen,ar_fopen)):
  if i>=100000: break
  if i%20000==0: print(i)
  en_line,ar_line=pair
  en_line,ar_line=en_line.strip("\n\r\t "),ar_line.strip("\n\r\t ")
  src_sents.append(en_line)
  trg_sents.append(ar_line)

ar_fopen.close()
en_fopen.close()
print(len(src_sents),len(src_sents))

0
20000
40000
60000
80000
100000 100000


#Start indexing

In [11]:
src_indexing_obj=indexing(src_sents,tok,stop_words=["the","of"],ignore_punc=True)
print("finihsed indexing source sentences")
trg_indexing_obj=indexing(trg_sents,cur_ar_tok_fn,ignore_ar_pre_suf=False,ignore_punc=True)
print("finihsed indexing target sentences")

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
finihsed indexing source sentences
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
finihsed indexing target sentences


#Start aligning

In [21]:
src_list=src_indexing_obj.all_tok_sentences
trg_list=trg_indexing_obj.all_tok_sentences
src_inv_dict=src_indexing_obj.inv_index
trg_inv_dict=trg_indexing_obj.inv_index

def get_tok_loc(tok_list):
  tok_locs_list=[]
  for i_,cur_tok in enumerate(tok_list):
    tok_locs_list.append(((cur_tok,),(i_,i_)))
    if i_<len(tok_list)-1: tok_locs_list.append(((cur_tok,tok_list[i_+1]), (i_,i_+1)))
  tok_locs_list.sort(key=lambda x:x[0])
  grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(tok_locs_list,lambda x:x[0])]
  return grouped

test_i=12000
for src0,trg0 in zip(src_list[test_i:test_i+10],trg_list[test_i:test_i+10]):
  print(src0)
  print(trg0)
  src0=['<s>']+src0+['</s>']
  trg0=['<s>']+trg0+['</s>']
  src_grouped=get_tok_loc(src0)
  trg_grouped=get_tok_loc(trg0)
  src_side,trg_side=[],[]

  for s_toks,s_locs_grp in src_grouped:
    src_indexes0= rtrv_idx(s_toks,src_inv_dict)
    src_side.append((s_toks,s_locs_grp,src_indexes0))
    #print(s_toks,len(src_indexes0))
  for t_toks,t_locs_grp in trg_grouped:
    #print(t_toks)
    trg_indexes0= rtrv_idx(t_toks,trg_inv_dict)
    trg_side.append((t_toks,t_locs_grp,trg_indexes0))
    #print(t_toks,len(trg_indexes0))
  matching_list=[]
  all_elements=[]
  for s_toks,s_locs_grp,src_indexes0 in src_side:
    s_toks_str=" ".join(s_toks)
    #continue
    #print(s_toks,s_locs_grp,len(src_indexes0))
    for t_toks,t_locs_grp,trg_indexes0 in trg_side:
      cur_ratio=0
      t_toks_str=" ".join(t_toks)
      if s_toks==t_toks: cur_ratio=1
      else: 
        #cur_ratio=idx_match(src_indexes0,trg_indexes0)
        if len(src_indexes0)==0 or len(trg_indexes0)==0: continue #cur_ratio= 0
        intersection=len(list(set(src_indexes0).intersection(set(trg_indexes0))))
        if intersection<10: continue
        union=len(src_indexes0)+len(trg_indexes0)-intersection
        if union==0: continue #cur_ratio= 0
        cur_ratio=float(intersection)/union
        #print(s_toks, t_toks,intersection,cur_ratio)
      if cur_ratio==0: continue
      for sl0 in s_locs_grp:
        for tl0 in t_locs_grp:
          cur_el=(sl0,tl0)
          all_elements.append((cur_el,s_toks_str,t_toks_str,cur_ratio))

      matching_list.append((s_toks,t_toks,cur_ratio))
  all_elements.sort(key=lambda x:(x[0][0][0],-x[-1]))
  all_elements_grouped=[(key,list(group)) for key,group in groupby(all_elements,lambda x:x[0][0][0])]
  el_start_dict={}
  for k0,el_grp in all_elements_grouped:
    el_start_dict[k0]=el_grp[:10]
  
  for inc0 in range(len(src0)):
    for i0 in range(len(src0)):
      i1=i0+inc0
      if i1>len(src0)-1: continue
      if inc0>0: continue
      print(i0,i1)
      for key0,vals0 in el_start_dict.items():
        

    # for el0 in el_grp[:10]:
    #   print(k0,el0)

  # matching_list.sort(key=lambda x:-x[-1])
  # used_src,used_trg=[],[]
  # for m in matching_list:
  #   src0,trg0,ratio=m
  #   if src0 in used_src: continue
  #   if trg0 in used_trg: continue
  #   used_src.append(src0)
  #   used_trg.append(trg0)
  #   print(m)


  print("========")

['302', 'as', 'far', 'as', 'development', 'new', 'armed', 'forces', 'doctrine', 'provided', 'for', 'by', 'peace', 'agreements', 'is', 'concerned', 'doctrine', 'command', 'armed', 'forces', 'has', 'been', 'working', 'out', 'details', 'broad', 'outlines', 'laid', 'down', 'under', 'constitutional', 'reform']
['302', 'وـ', 'فيما', 'يتعلق', 'بـ', 'وضع', 'عقيدة', 'جديدة', 'لـ', 'قوات', 'مسلحة', 'مما', 'ورد', 'في', 'اتفاقات', 'سلم', 'يلاحظ', 'أن', 'قيادة', 'عقائدية', 'لـ', 'قوات', 'مسلحة', 'مافتئت', 'تعمل', 'على', 'وضع', 'تفاصيل', 'خطوط', 'أساسية', 'محددة', 'في', 'إطار', 'اصلاح', 'دستوري']
0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
['303', 'it', 'should', 'be', 'recalled', 'however', 'that', 'under', 'terms', 'peace', 'agreements', 'new', 'armed', 'forces', 'doctrine', 'must', 'be', 'published', 'this', 'obligation', 'has', 'yet', 'to', 'be', 'fulfilled'

#OLD

In [ ]:
test_indexing_obj.inv_index

{'ال_أشخاص': [(0, 3)],
 'ال_عاملين': [(0, 4)],
 'ال_مكتب': [(1, 2)],
 'ال_منظمات': [(0, 1)],
 'ـها': [(0, 6)],
 'في': [(0, 5), (1, 1)],
 'معظم': [(0, 0)],
 'وـ': [(0, 2)],
 'ونحن': [(1, 0)]}

In [ ]:
#Testing the function
from code_utils.align_utils import *
from code_utils.arabic_lib import *
from code_utils.general import *
from itertools import groupby
from collections import Counter
ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})

class indexing: #get a list of sentences, outputs indexes
  def __init__(self,raw_sentences0,tok_function=tok, lang="en",stop_words=[], ignore_punc=False,ignore_ar_pre_suf=False,remove_al=True,index_words=True,lower=True,stemming=False):
    self.fwd_index=[]
    self.all_tok_sentences=[]
    self.all_tok_original_sentences=[]
    self.all_mappings=[]
    self.inv_index={}
    for sent_i,sent0 in enumerate(raw_sentences0):
      #if lower: sent0=sent0.lower()
      tokens=tok_function(sent0)
      tokens_original=list(tokens)
      tokens_copy_enum=[(i,v) for i,v in enumerate(tokens_original)]
      self.all_tok_original_sentences.append(tokens_original)
      
      #if lang=="ar": tokens=tok_ar(tokens,count_dict) #tok_ar(tokens)
      if ignore_punc: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not is_punct(v)]
      if ignore_ar_pre_suf: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not v.startswith("ـ") and not v.endswith("ـ")]
      if stop_words: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not v.lower() in stop_words]
      if remove_al: tokens_copy_enum=[(i,v.replace("ال_","")) for i,v in tokens_copy_enum if not v.lower() in stop_words]
      tokens=[v[1] for v in tokens_copy_enum]
      cur_mapping=[v[0] for v in tokens_copy_enum] #mapping of token locations to original after excluding some elements
      self.all_mappings.append(cur_mapping)
      
      tokens_lower=[v.lower() for v in tokens]
      if lower: enumerated_toks=[(v,sent_i,s_i) for s_i,v in enumerate(tokens_lower)]
      else: enumerated_toks=[(v,sent_i,s_i) for s_i,v in enumerate(tokens)]
      self.all_tok_sentences.append(tokens)
      self.fwd_index.extend(enumerated_toks)
    self.fwd_index.sort()
    grouped=[(key,[v[1:] for v in list(group)]) for key,group in groupby(self.fwd_index,lambda x:x[0])]
    self.inv_index=dict(iter(grouped))

def cur_ar_tok_fn(text):
  cur_toks=tok(text)
  return tok_ar(cur_toks,ar_counter_dict)
sents=["here is it, I'll tell you","what's this?","When I go, I'll tell you!"]
sents=["معظم المنظمات والأشخاص العاملين فيها","ونحن في المكتب","وهذه هي المشكلة الأكبر فيها وفي معظم الأمور الأخرى"]
#test_indexing_obj=indexing(sents)
test_indexing_obj=indexing(sents,cur_ar_tok_fn,ignore_ar_pre_suf=True)
print(test_indexing_obj.all_tok_sentences)
print(test_indexing_obj.all_tok_original_sentences)
print(test_indexing_obj.all_mappings)

[['معظم', 'منظمات', 'أشخاص', 'عاملين', 'في'], ['ونحن', 'في', 'مكتب'], ['هذه', 'هي', 'مشكلة', 'أكبر', 'في', 'في', 'معظم', 'أمور', 'أخرى']]
[['معظم', 'ال_منظمات', 'وـ', 'ال_أشخاص', 'ال_عاملين', 'في', 'ـها'], ['ونحن', 'في', 'ال_مكتب'], ['وـ', 'هذه', 'هي', 'ال_مشكلة', 'ال_أكبر', 'في', 'ـها', 'وـ', 'في', 'معظم', 'ال_أمور', 'ال_أخرى']]
[[0, 1, 3, 4, 5], [0, 1, 2], [1, 2, 3, 4, 5, 8, 9, 10, 11]]
